In [5]:
import os
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datetime import datetime

class MedicalSummarizer:
    def __init__(self, model_name='t5-large'):
        """
        Initialize the Medical Summarizer with a specified T5 model.
        """
        self.model_name = model_name
        self.model = None
        self.tokenizer = None
        self.load_model_and_tokenizer()

    def load_model_and_tokenizer(self):
        """
        Load the T5 model and tokenizer directly from Hugging Face.
        """
        try:
            print(f"Loading model: {self.model_name}")

            # Load tokenizer
            self.tokenizer = T5Tokenizer.from_pretrained(self.model_name)
            print("Tokenizer loaded successfully.")

            # Load model
            self.model = T5ForConditionalGeneration.from_pretrained(self.model_name)
            print("Model loaded successfully.")

            # Move to GPU if available
            if torch.cuda.is_available():
                self.model = self.model.to('cuda')
                print("Model moved to CUDA.")

        except Exception as e:
            print(f"An error occurred while loading model and tokenizer: {e}")
            raise

    def generate_summary(self, input_text, max_length=300, min_length=100):
        """
        Generate a summary for the input text and save it to a file.

        Args:
            input_text (str): Text to summarize
            max_length (int): Maximum length of the summary
            min_length (int): Minimum length of the summary

        Returns:
            str: Path to the saved summary file
        """
        try:
            # Prepare input
            input_text = f"summarize: {input_text}"

            # Tokenize input
            inputs = self.tokenizer.encode(
                input_text,
                return_tensors='pt',
                max_length=512,
                truncation=True
            )

            # Move inputs to GPU if available
            if torch.cuda.is_available():
                inputs = inputs.to('cuda')

            # Generate summary
            summary_ids = self.model.generate(
                inputs,
                max_length=max_length,
                min_length=min_length,
                num_beams=4,
                early_stopping=True,
                no_repeat_ngram_size=2
            )

            # Decode summary
            summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Create summaries directory if it doesn't exist
            os.makedirs('summaries', exist_ok=True)

            # Generate filename with timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"summaries/medical_summary_{timestamp}.txt"

            # Save summary to file
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(summary)

            print(f"Summary saved to {filename}")
            return filename

        except Exception as e:
            print(f"Error during summary generation: {e}")
            return None

def main():
    # Create summarizer instance
    summarizer = MedicalSummarizer()

    # Interactive loop
    while True:
        # Prompt user for input
        print("\n--- Medical Text Summarization Tool ---")
        print("Enter the medical text you want to summarize.")
        print("(Press Enter twice or type 'quit' to exit)")

        # Collect multi-line input
        input_lines = []
        while True:
            line = input()
            if line == '' or line.lower() == 'quit':
                break
            input_lines.append(line)

        # Check if user wants to quit
        if not input_lines or input_lines[0].lower() == 'quit':
            print("Exiting the summarization tool.")
            break

        # Combine input lines
        full_text = ' '.join(input_lines)

        try:
            # Generate and save summary
            summary_file = summarizer.generate_summary(full_text)

            # Option to view summary
            if summary_file:
                print("\n--- Summary Generated ---")
                with open(summary_file, 'r', encoding='utf-8') as f:
                    print(f.read())

        except Exception as e:
            print(f"An error occurred: {e}")

if __name__ == '__main__':
    main()


Loading model: t5-large
Tokenizer loaded successfully.
Model loaded successfully.
Model moved to CUDA.

--- Medical Text Summarization Tool ---
Enter the medical text you want to summarize.
(Press Enter twice or type 'quit' to exit)
# Patient Medical History: Emily Rodriguez  ## Patient Profile - Name: Emily Rodriguez - Age: 52 - Gender: Female - Occupation: High School Chemistry Teacher - Height: 5'6" - Weight: 168 lbs  ## Comprehensive Medical History  ### Chronic Conditions 1. **Rheumatoid Arthritis (RA)**    - Diagnosed at age 38    - Currently managed with methotrexate and hydroxychloroquine    - Experiences periodic flare-ups affecting hands and knees    - Regular rheumatology follow-ups every 3 months    - Current disease activity score indicates moderate inflammation  2. **Type 2 Diabetes**    - Diagnosed at age 45    - Initial HbA1c of 7.8%, currently maintained at 6.5%    - Managed with oral medications (Metformin 1000mg twice daily)    - Implements strict dietary modificatio

In [8]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

class LanguageTranslator:
    def __init__(self):
        """
        Initialize translators using Meta's NLLB model for Indian languages.
        """
        self.language_models = {
            'hindi': 'facebook/nllb-200-distilled-600M',
            'marathi': 'facebook/nllb-200-distilled-600M',
            'gujarati': 'facebook/nllb-200-distilled-600M'
        }
        self.language_codes = {
            'hindi': 'hin_Deva',
            'marathi': 'mar_Deva',
            'gujarati': 'guj_Deva'
        }
        self.loaded_models = {}

    def load_translation_model(self):
        """
        Load the NLLB translation model.

        Returns:
            dict: Tokenizer and model for translation
        """
        model_name = 'facebook/nllb-200-distilled-600M'

        try:
            # Check if model is already loaded
            if model_name in self.loaded_models:
                return self.loaded_models[model_name]

            # Print loading message
            print("Loading NLLB translation model...")

            # Load tokenizer and model
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

            # Move to GPU if available
            if torch.cuda.is_available():
                model = model.to('cuda')
                print("Model moved to CUDA.")

            # Cache the loaded model
            self.loaded_models[model_name] = {
                'tokenizer': tokenizer,
                'model': model
            }

            print("NLLB translation model loaded successfully.")
            return self.loaded_models[model_name]

        except Exception as e:
            print(f"Error loading translation model: {e}")
            return None

    def translate_text(self, text, target_language):
        """
        Translate input text to the specified language.

        Args:
            text (str): Text to translate
            target_language (str): Target language code

        Returns:
            str: Translated text
        """
        # Validate and normalize language input
        target_language = target_language.lower()

        # Validate language
        if target_language not in self.language_codes:
            raise ValueError(f"Unsupported language: {target_language}")

        # Load translation model
        model_data = self.load_translation_model()

        if not model_data:
            return f"Failed to load translation model"

        # Extract tokenizer and model
        tokenizer = model_data['tokenizer']
        model = model_data['model']

        try:
            # Prepare input with NLLB-specific formatting
            inputs = tokenizer(
                text,
                return_tensors='pt',
                max_length=512,
                truncation=True
            )

            # Move to GPU if available
            if torch.cuda.is_available():
                inputs = {k: v.to('cuda') for k, v in inputs.items()}

            # Generate translation
            outputs = model.generate(
                **inputs,
                forced_bos_token_id=tokenizer.convert_tokens_to_ids(self.language_codes[target_language]),  # FIXED
                max_length=1024,
                num_beams=4,
                early_stopping=True
            )

            # Decode translation
            translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

            return translated_text

        except Exception as e:
            return f"Translation error: {e}"


def main():
    # Create translator instance
    translator = LanguageTranslator()

    while True:
        # Display menu
        print("\n--- Advanced Medical Text Translation Tool ---")
        print("1. Translate to Hindi")
        print("2. Translate to Marathi")
        print("3. Translate to Gujarati")
        print("4. Exit")

        # Get user choice
        try:
            choice = input("Enter your choice (1-4): ").strip()

            # Exit condition
            if choice == '4':
                print("Exiting translation tool.")
                break

            # Validate choice
            if choice not in ['1', '2', '3']:
                print("Invalid choice. Please try again.")
                continue

            # Map choice to language
            language_map = {
                '1': 'hindi',
                '2': 'marathi',
                '3': 'gujarati'
            }
            target_language = language_map[choice]

            # Prompt for input text
            print("\nEnter the text to translate (Press Enter twice to finish):")
            input_lines = []
            while True:
                line = input()
                if line == '':
                    break
                input_lines.append(line)

            # Combine input lines
            input_text = ' '.join(input_lines)

            # Translate text
            translated_text = translator.translate_text(input_text, target_language)

            # Display translated text
            print(f"\n--- Translated to {target_language.capitalize()} ---")
            print(translated_text)

            # Save option
            save_choice = input("\nDo you want to save the translation? (y/n): ").lower().strip()
            if save_choice == 'y':
                # Create translations directory if it doesn't exist
                os.makedirs('translations', exist_ok=True)

                # Generate filename
                filename = f"translations/translation_{target_language}_{len(os.listdir('translations'))+1}.txt"

                # Save translation
                with open(filename, 'w', encoding='utf-8') as f:
                    f.write(translated_text)

                print(f"Translation saved to {filename}")

        except Exception as e:
            print(f"An error occurred: {e}")

if __name__ == '__main__':
    main()



--- Advanced Medical Text Translation Tool ---
1. Translate to Hindi
2. Translate to Marathi
3. Translate to Gujarati
4. Exit
Enter your choice (1-4): 1

Enter the text to translate (Press Enter twice to finish):
# Patient Medical History: Emily Rodriguez ## Age: 52 - Gender: Female- Occupation: high school chemistry teacher 168 lbs . # Chronic Conditions: **Rheumatoid Arthritis (RA) **Type 2 diabetes** --- Diagnosed at age 45 -- Initial HbA1c of 7.8%, currently maintained at 6.5% -- Implements strict dietary modifications -- Monitors blood glucose levels twice daily -- Mild peripheral neuropathy in left foot ### Respiratory Complication

Loading NLLB translation model...
Model moved to CUDA.
NLLB translation model loaded successfully.

--- Translated to Hindi ---
# रोगी चिकित्सा इतिहासः एमिली रोड्रिगेज ## आयुः 52 - लिंगः महिला- व्यवसायः हाई स्कूल के रसायन विज्ञान शिक्षक 168 पाउंड . # पुरानी स्थितियांः ** रेमटोइड गठिया (आरए) ** टाइप 2 मधुमेह** --- 45 वर्ष की आयु में निदान किया गया -- 

In [10]:
import spacy
from collections import Counter
import re
from typing import List, Dict, Tuple
from sklearn.feature_extraction.text import TfidfVectorizer


class MedicalKeywordExtractor:
    def __init__(self):
        # Load English language model
        self.nlp = spacy.load("en_core_web_sm")

        # Add medical terms to the vocabulary
        self.medical_terms = {
            "symptoms": ["pain", "fever", "headache", "nausea", "vomiting", "fatigue", "cough", "shortness of breath"],
            "conditions": ["diabetes", "hypertension", "asthma", "arthritis", "cancer", "infection", "inflammation"],
            "medications": ["antibiotics", "insulin", "antidepressants", "painkillers", "steroids"],
            "procedures": ["surgery", "biopsy", "x-ray", "mri", "ct scan", "blood test"],
            "body_parts": ["heart", "lungs", "brain", "liver", "kidneys", "stomach", "bones"],
            "measurements": ["blood pressure", "temperature", "heart rate", "weight", "height"]
        }

        # Add custom patterns for medical terms
        if "entity_ruler" not in self.nlp.pipe_names:
            ruler = self.nlp.add_pipe("entity_ruler", before="ner")
        else:
            ruler = self.nlp.get_pipe("entity_ruler")

        patterns = []
        for category, terms in self.medical_terms.items():
            for term in terms:
                patterns.append({"label": category.upper(), "pattern": term})
        ruler.add_patterns(patterns)

    def preprocess_text(self, text: str) -> str:
        """Clean and preprocess the input text."""
        # Convert to lowercase
        text = text.lower()
        # Remove special characters but keep medical terms
        text = re.sub(r'[^\w\s]', ' ', text)
        # Remove extra whitespace
        text = ' '.join(text.split())
        return text

    def extract_keywords(self, text: str, top_n: int = 10) -> List[Tuple[str, float]]:
        """
        Extract keywords from the input text using multiple methods.
        Returns a list of (keyword, score) tuples.
        """
        # Preprocess text
        text = self.preprocess_text(text)

        # Method 1: TF-IDF
        vectorizer = TfidfVectorizer(max_features=100)
        tfidf_matrix = vectorizer.fit_transform([text])
        feature_names = vectorizer.get_feature_names_out()
        tfidf_scores = tfidf_matrix.toarray()[0]

        # Method 2: Named Entity Recognition
        doc = self.nlp(text)
        ner_entities = [(ent.text, 1.0) for ent in doc.ents if not re.match(r'^\d+$', ent.text)]  # Ignore numbers

        # Method 3: Part of Speech tagging for medical terms
        pos_keywords = []
        for token in doc:
            if token.pos_ in ['NOUN', 'ADJ'] and len(token.text) > 2 and not token.text.isdigit():
                pos_keywords.append((token.text, 0.8))

        # Combine all keywords and their scores
        all_keywords = {}

        # Add TF-IDF keywords
        for word, score in zip(feature_names, tfidf_scores):
            if score > 0 and not re.match(r'^\d+$', word):  # Ignore numbers
                all_keywords[word] = score

        # Add NER entities
        for word, score in ner_entities:
            all_keywords[word] = max(all_keywords.get(word, 0), score)

        # Add POS-based keywords
        for word, score in pos_keywords:
            all_keywords[word] = max(all_keywords.get(word, 0), score)

        # Sort keywords by score and return top N
        sorted_keywords = sorted(all_keywords.items(), key=lambda x: x[1], reverse=True)
        return sorted_keywords[:top_n]


    def categorize_keywords(self, keywords: List[Tuple[str, float]]) -> Dict[str, List[Tuple[str, float]]]:
        """
        Categorize the extracted keywords into medical categories.
        """
        categorized = {category: [] for category in self.medical_terms.keys()}
        categorized['other'] = []

        medical_terms_set = {term for category in self.medical_terms.values() for term in category}

        for keyword, score in keywords:
            if keyword in medical_terms_set:
                for category, terms in self.medical_terms.items():
                    if keyword in terms:
                        categorized[category].append((keyword, score))
                        break
            else:
                categorized['other'].append((keyword, score))

        return categorized


class MedicalSummaryInput:
    def __init__(self):
        self.summary = ""

    def get_input(self):
        """Collects medical summary from user input (until an empty line is entered)."""
        print("Enter your medical summary (press Enter on an empty line when done):")
        while True:
            line = input()
            if line.strip() == "":
                break
            self.summary += line + " "

        if not self.summary.strip():
            print("No summary provided. Exiting.")
            return None
        return self.summary


class MedicalKeywordProcessor:
    def __init__(self):
        self.extractor = MedicalKeywordExtractor()
        self.input_handler = MedicalSummaryInput()

    def process(self):
        # Get medical summary input from user
        summary = self.input_handler.get_input()
        if not summary:
            return

        # Extract keywords
        keywords = self.extractor.extract_keywords(summary)

        # Categorize keywords
        categorized_keywords = self.extractor.categorize_keywords(keywords)

        # Print results
        print("\nExtracted Keywords:")
        print("-" * 50)
        for category, words in categorized_keywords.items():
            if words:  # Only print categories that have keywords
                print(f"\n{category.upper()}:")
                for word, score in words:
                    print(f"  - {word} (score: {score:.4f})")


def main():
    processor = MedicalKeywordProcessor()
    processor.process()


if __name__ == "__main__":
    main()

Enter your medical summary (press Enter on an empty line when done):
# Patient Medical History: Emily Rodriguez ## Age: 52 - Gender: Female- Occupation: high school chemistry teacher 168 lbs . # Chronic Conditions: **Rheumatoid Arthritis (RA) **Type 2 diabetes** --- Diagnosed at age 45 -- Initial HbA1c of 7.8%, currently maintained at 6.5% -- Implements strict dietary modifications -- Monitors blood glucose levels twice daily -- Mild peripheral neuropathy in left foot ### Respiratory Complication


Extracted Keywords:
--------------------------------------------------

CONDITIONS:
  - arthritis (score: 1.0000)
  - diabetes (score: 1.0000)

OTHER:
  - age 45 (score: 1.0000)
  - 7 8 (score: 1.0000)
  - 6 5 (score: 1.0000)
  - age (score: 0.8000)
  - blood (score: 0.8000)
  - chemistry (score: 0.8000)
  - chronic (score: 0.8000)
  - complication (score: 0.8000)
